# Elektron WebSocket API MRN Example with Python

## Machine Readable News Overview

Refinitiv Machine Readable News (MRN) is an advanced service for automating the consumption and systematic analysis of news. It delivers deep historical news archives, ultra-low latency structured news and news analytics directly to your applications. This enables algorithms to exploit the power of news to seize opportunities, capitalize on market inefficiencies and manage event risk.

MRN aims for replacing the legacy News 2K (N2_UBMS and N2_STORY).

### MRN Data behavior
MRN is published over Elektron using an Open Message Model (OMM) envelope in News Text Analytics domain messages. The Real-time News content set is made available over MRN_STORY RIC. The content data is contained in a FRAGMENT field that has been compressed, and potentially fragmented across multiple messages, in order to reduce bandwidth and message size.

A FRAGMENT field has a different data type based on a connection type:
* RSSL connection (ESDK [C++](https://developers.refinitiv.com/elektron/elektron-sdk-cc)/[Java](https://developers.refinitiv.com/elektron/elektron-sdk-java)): BUFFER type
* WebSocket connection: Base64 ascii string

The data goes through the following series of transformations:

1. The core content data is a UTF-8 JSON string
2. This JSON string is compressed using gzip
3. The compressed JSON is split into a number of fragments (BUFFER or Base64 ascii string) which each fit into a single update message
4. The data fragments are added to an update message as the FRAGMENT field value in a FieldList envelope


<img src="images/mrn_process.png"/>

Therefore, in order to parse the core content data, the application will need to reverse this process. The WebSocket application also need to convert a received Base64 string in a FRAGMENT field to bytes data before further process this field.

### MRN Data model

Five fields, as well as the RIC itself, are necessary to determine whether the entire item has been received in its various fragments and how to concatenate the fragments to reconstruct the item:
* MRN_SRC: identifier of the scoring/processing system that published the FRAGMENT
* GUID: a globally unique identifier for the data item. All messages for this data item will have the same GUID values.
* FRAGMENT: compressed data item fragment, itself
* TOT_SIZE: total size in bytes of the fragmented data
* FRAG_NUM: sequence number of fragments within a data item. This is set to 1 for the first fragment of each item published and is incremented for each subsequent fragment for the same item.

A single MRN data item publication is uniquely identified by the combination of RIC, MRN_SRC, and GUID.

#### Fragmentation
For a given RIC-MRN_SRC-GUID combination, when a data item requires only a single message, then TOT_SIZE will equal the number of bytes in the FRAGMENT and FRAG_NUM will be 1.

When multiple messages are required, then the data item can be deemed as fully received once the sum of the number of bytes of each FRAGMENT equals TOT_SUM. The consumer will also observe that all FRAG_NUM range from 1 to the number of the fragment, with no intermediate integers skipped. In other words, a data item transmitted over three messages will contain FRAG_NUM values of 1, 2 and 3.

#### Compression
The FRAGMENT field is compressed with gzip compression, thus requiring the consumer to decompress to reveal the JSON plain-text data in that FID.

When an MRN data item is sent in multiple messages, all the messages must be received and their FRAGMENTs concatenated before being decompressed. In other words, the FRAGMENTs should not be decompressed independently of each other.

The decompressed output is encoded in UTF-8 and formatted as JSON.

Please see a full documentation of this example application in [this article](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).

If you are not familiar with MRN concept, please visit the following resources which will give you a full explanation of the MRN data model and implementation logic:
* [Webinar Recording: Introduction to Machine Readable News](https://developers.refinitiv.com/news#news-accordion-nid-12045)
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).

In [36]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install requests and websocket-client packages in a current Jupyter kernal\n

import sys

# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [37]:

import time
import getopt
import socket
import json
import websocket
import threading
from threading import Thread, Event
import base64
import zlib
import requests

### EDP Credentials

Please put in your EDP credentials and client ID here.

In [44]:
# EDP Credentials

user = ''
password = ''
client_id = ''

In [ ]:
# EDP connection variables

token = ''
token_api_endpoint = 'https://api.refinitiv.com/auth/oauth2/v1/token'
hostname = 'apac-3.pricing.streaming.edp.thomsonreuters.com'
port = '443'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [45]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False
_news_envelopes = []

# keeps decompress news JSON messaage
_news_messages = []

### EDP Token retrival

In order to connect to ERT on Cloud, you need to retrieve token from the Token API first. This part of the code will take care of then token retrieval for you.

In [46]:
# EDP Token Retrieval

def get_token():
    print('Retrieve Token')
    data = {
        'client_id': client_id,
        'grant_type': 'password',
        'password': password,
        'scope': 'trapi',
        'takeExclusiveSignOnControl': True,
        'username': user,
    }
    #print(data)
    r = requests.post(token_api_endpoint,
        data=data,
        verify=True)
    auth_json = r.json()
    #print(json.dumps(auth_json, sort_keys=True, indent=2, separators=(',', ':')))
    response = {
        'token': '',
        'error': ''
    }
    if ('access_token' in auth_json):
        response['token'] = auth_json['access_token']
    else:
        response['error'] = auth_json['error_description']
    return response

### MRN Process Code

The MRN data can be subscribed with the *NewsTextAnalytics* domain and MRN-specific RIC name as following:
- *MRN_STORY*: Real-time News
- *MRN_TRNA*: News Analytics: Company and C&E assets
- *MRN_TRNA_DOC*: News Analytics: Macroeconomic News & events
- *MRN_TRSI*: News Sentiment Indices

 

In [47]:
# MRN variables

mrn_domain = 'NewsTextAnalytics'
mrn_item = 'MRN_STORY'

def send_mrn_request(ws):
    """ Create and send MRN request """
    mrn_req_json = {
        'ID': 2,
        "Domain": mrn_domain,
        'Key': {
            'Name': mrn_item
        }
    }

    ws.send(json.dumps(mrn_req_json))
    print("SENT:")
    print(json.dumps(mrn_req_json, sort_keys=True, indent=2, separators=(',', ':')))

### Initial Refresh Message
The Initial Refresh response does not contain any NTA data, all the fields related to news item and fragment are empty or 0. It contains only the relevant feed related or other static Fields. 

The application can just print out each incoming field data in a console for informational purpose or just ignore it.

In [48]:
# Process FieldList, Refresh and Status messages.

def decodeFieldList(fieldList_dict):
    for key, value in fieldList_dict.items():
        print("Name = %s: Value = %s" % (key, value))

def processRefresh(ws, message_json):

    print("RECEIVED: Refresh Message")
    decodeFieldList(message_json["Fields"])

def processStatus(ws, message_json):  # process incoming status message
    print("RECEIVED: Status Message")
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

### MRN News Update messages Process Code

The updates contain only fields related to the item and the fragment. They do not contain any of the static or per-feed fields. The updates are not cached or conflated.

#### First Update
The first update contains all the fields related to the item and the first fragment, subsequent updates only contain the fields relating to the fragment they contain. The FRAG_NUM FID is set to 1 for the first Update of each item and is incremented in each subsequent Update for that item. This allows you to you to detect a missing fragment (and ensure correct order of the fragments for re-assembly). 


#### Subsequent Update and Multi Fragment Items
The subsequent update contains the fields necessary to identify the MRN data item, the order of this fragment among all the fragments for this item, and the fragment itself. The other point to note is that (for a Multi fragment item), Update messages with FRAG_NUM >1 will have fewer FIDs as the metadata is included in the first Update message (FRAG_NUM=1) for that item

#### News Fragments simple handle logic


<img src="images/mrn_flow_reconstruct.png"/>


In [49]:
def processMRNUpdate(ws, message_json):  # process incoming News Update messages

    fields_data = message_json["Fields"]
    # Dump the FieldList first (for informational purposes)
    # decodeFieldList(message_json["Fields"])

    # declare variables
    tot_size = 0
    guid = None

    try:
        # Get data for all requried fields
        fragment = base64.b64decode(fields_data["FRAGMENT"])
        frag_num = int(fields_data["FRAG_NUM"])
        guid = fields_data["GUID"]
        mrn_src = fields_data["MRN_SRC"]

        #print("GUID  = %s" % guid)
        #print("FRAG_NUM = %d" % frag_num)
        #print("MRN_SRC = %s" % mrn_src)

        if frag_num > 1:  # We are now processing more than one part of an envelope - retrieve the current details
            guid_index = next((index for (index, d) in enumerate(
                _news_envelopes) if d["guid"] == guid), None)
            envelop = _news_envelopes[guid_index]
            if envelop and envelop["data"]["mrn_src"] == mrn_src and frag_num == envelop["data"]["frag_num"] + 1:
                print("process multiple fragments for guid %s" %
                      envelop["guid"])

                #print("fragment before merge = %d" % len(envelop["data"]["fragment"]))

                # Merge incoming data to existing news envelop and getting FRAGMENT and TOT_SIZE data to local variables
                fragment = envelop["data"]["fragment"] = envelop["data"]["fragment"] + fragment
                envelop["data"]["frag_num"] = frag_num
                tot_size = envelop["data"]["tot_size"]
                print("TOT_SIZE = %d" % tot_size)
                print("Current FRAGMENT length = %d" % len(fragment))

                # The multiple fragments news are not completed, waiting.
                if tot_size != len(fragment):
                    return None
                # The multiple fragments news are completed, delete assoiclate GUID envelop
                elif tot_size == len(fragment):
                    del _news_envelopes[guid_index]
            else:
                print("Error: Cannot find fragment for GUID %s with matching FRAG_NUM or MRN_SRC %s" % (
                    guid, mrn_src))
                return None
        else:  # FRAG_NUM = 1 The first fragment
            tot_size = int(fields_data["TOT_SIZE"])
            print("FRAGMENT length = %d" % len(fragment))
            # The fragment news is not completed, waiting and add this news data to envelop object.
            if tot_size != len(fragment):
                print("Add new fragments to news envelop for guid %s" % guid)
                _news_envelopes.append({  # the envelop object is a Python dictionary with GUID as a key and other fields are data
                    "guid": guid,
                    "data": {
                        "fragment": fragment,
                        "mrn_src": mrn_src,
                        "frag_num": frag_num,
                        "tot_size": tot_size
                    }
                })
                return None

        # News Fragment(s) completed, decompress and print data as JSON to console
        if tot_size == len(fragment):
            print("decompress News FRAGMENT(s) for GUID  %s" % guid)
            decompressed_data = zlib.decompress(fragment, zlib.MAX_WBITS | 32)
            
            json_news = json.loads(decompressed_data)
            _news_messages.append(json_news)
            print("News = %s" % json_news)

    except KeyError as keyerror:
        print('KeyError exception: ', keyerror)
    except IndexError as indexerror:
        print('IndexError exception: ', indexerror)
    except binascii.Error as b64error:
        print('base64 decoding exception:', b64error)
    except zlib.error as error:
        print('zlib decompressing exception: ', error)
    # Some console environments like Windows may encounter this unicode display as a limitation of OS
    except UnicodeEncodeError as encodeerror:
        print("UnicodeEncodeError exception. Cannot decode unicode character for %s in this enviroment: " %
              guid, encodeerror)
    except Exception as e:
        print('exception: ', sys.exc_info()[0])

### JSON-OMM Process functions

In [50]:
def process_message(ws, message_json):
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if "Domain" in message_json:
            message_domain = message_json["Domain"]
            if message_domain == "Login":
                process_login_response(ws, message_json)
            elif message_domain:
                processRefresh(ws, message_json)
    elif message_type == "Update":
        if "Domain" in message_json and message_json["Domain"] == mrn_domain:
            processMRNUpdate(ws, message_json)
    elif message_type == "Status":
        processStatus(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))


def process_login_response(ws, message_json):
    """ Send item request """
    send_mrn_request(ws)


def send_login_request(ws):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': 1,
        "Domain": 'Login',
        'Key': {
            #'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            },
            'NameType': 'AuthnToken'
        }
    }

    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position
    login_json['Key']['Elements']['AuthenticationToken'] = token

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

### WebSocket Process functions

In [ ]:
def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws)

if __name__ == "__main__":
    # Get token
    token_result = get_token()
    if (token_result['token'] != ''):
        # Success
        token = token_result['token']
    else:
        # Failed
        print(token_result['error'])
        sys.exit(2)
    # Start websocket handshake
    ws_address = "wss://{}:{}/WebSocket".format(hostname, port)
    print("Connecting to WebSocket " + ws_address + " ...")
    web_socket_app = websocket.WebSocketApp(ws_address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    #wst = threading.Thread(target=web_socket_app.run_forever)
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()

    time.sleep(30)
    web_socket_app.close()

Retrieve Token
Connecting to WebSocket wss://apac-3.pricing.streaming.edp.thomsonreuters.com:443/WebSocket ...
WebSocket successfully connected!
SENT:
{
  "Domain":"Login",
  "ID":1,
  "Key":{
    "Elements":{
      "ApplicationId":"256",
      "AuthenticationToken":"eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6IkhQS2pEYUhiZmpLOVRfRHUxeGFJX1JaVmZfRjNDcHlEc0dTcDV5NWxCYVEifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwiUzh1YmFYcjBCZFNmWVZjb05zRkJBM056NDFCSVhGWlh5NVVBd2VFMEYtekh5TnY4WkNKdTgzMUFxNUE4UllGSjJvUEg3aDZBcVpJV3UzUGNKak94R3dDUG4yV2EtV1lmMC1IREV3eHRrdHphMXktSjF0WnRXREZfRWo1cnQzNWpnTnFQVTZudzdNaFBrSWctNEVFcTl5RHlpU0NuTXpILWc3VFNUWWNZSEx1cjQwbHJyczFpMmx0QkhrLXJ6SjlsTDhIaFJrNnRNS1dmald4cW1pNFBHSVBSdHZ4WnJ5ckVfeVFoVjhZQkk4Ym1RYnlTVGRSWHRzaHdoNDA2ZS1VMVF1V3NzdGYyMzcxRkcwZFlRdGFYNU1qOTVVb1pLZ21NSElveEFqSmRReGtQWFNpMzgxeThJTUVZV0NhWThibHNPeC1EWFNZNDl0RjRiRGtVc0c2MnFLbVkxdWhvLVBSQXdLV1FwUTluZFAtY1NobE04ZktsQkdUVTVNcTRaU0Y1UEx3bFdia3YxS3VYcXcxbndYa2lDSS1MTXZZbGx2WklNWHpFcUdHZWZHdkFINW1HeXc2OUduV

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-05-14",
      "FRAGMENT":"H4sIAAAAAAAC/62TS2/bMAyA7/sVhLBb40aSHcfRLe91ix+LnWJ9oVBqJXXr2pktpw2G/fdRcXLvYYBBfqRoiRSpP0Tm+iolghSR5nofXHPSIbJJM1U8qZqIOxJEIpoPyUOHrMv0gJHM8eWB0y5QhzMY+jCdJF3qIc/95L6wrDMLiHKpdQ1+uZU1DDjMwhHEupIZOr83xcBCwewOXF8l4SKeA67UqoZ1lsJX3r+0KWzKCnoUwnEI4AnO8Lu0Xc9sD3fguQN34DnMAc+jnu3a1IYHk8Lp4HlermUOw1xV2rLuC6xtk1W1HldKamXK5pRTi/Ys5iT0vH/Pdm8x8lnJNM8KhVHRYpgksQA2oJYVqHe4KatXYJSx/1zjJ0vE7DKT/blnj4+c0h5zuGY/4q5Ov4ULb3SRvoT9QX/mXKzGz7eV25+9Rx+//GRu/i5qLU2Hww32GHuby2LbyK0pVplrelN1jWZy2KGLo529nQyi1Yfu7nKZmbhdVe6zVFXoD2IRLZLYOJt1rKVucH4InqNFU8t1rnClbtYv6kkfB2soHPRMBTtKh7bK+HzBfk5QrwR3jmqMcmnGkGIHztxiwMVoOE5aGk+SmxOF/gmiZThp8SqYrFpaTIMWguW8hXA5WbZ0jMcb0fJVxep3Yx4CEaxDmmqLjC/A7pC9quqsLD4xR3+//AMZjiwhZAMAAA==",
      "FRAG_NUM":1,
      "GUID":"Pt2tvNV2__2005142t1KS/tdHOL8B+djO797F4+UChZr67FwPxXMTG",
      "MRN_SRC":

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-05-14",
      "FRAGMENT":"H4sIAAAAAAAC/41TS5OaQBC+51dYc8glPngqTtVWCtFdn0gA17hJyhpkQBJEAsOum1T+e3oYXHMMh/6+6ff00L8RSdksRBhlpveZBZ4boTYiVZjQ7EBLhL8g28FgGoNasDdio29tFJzDV4g+MpbjXi8Po25BK0aLsns4n3qkvGT0pbxil5T5x+ROUyV9eFCDSKLqQNfk99VdVWSYe5xS/E88ViRFknRZ2/PuNOti7cKvGTQQJUXJrIISRnnv3K0j6R1Z8yUDSxKW1e5AV57A80hJmCYZBS/rSLKYtpKsVVZByUjGEpK2juc0TLK4FRXnU2u1uX/o+M62a/JrJjz3dSz7vWhF+RXpu423cUez+D44fljGxuPQm/W3o93Sn0cv1s5VYsajM14DhriOYIwwqhTKVyTmrVB+iRMtSzj6rzmoFDgnp+aAGL2w3qWTpyTJOlFygVu2UV6cn5OQFmC3veYl8irwGGEVvBSCcgxXJQlSCha443d6YPUTjrCsGUNQjvhEG9QF6gL7wjw0ACZYrqVu1uCCfMDKtAa9lguQKyzbtsBPY4HbeYNCryoCpjXYmxr4ozhYl7SBpmmDvgpHF7+NnFvqb6iqdUOyYgjsy6K0JtcRhir1uz7/FxU8dwSO7IV3Y7Kg7nS9aqjnWO7Ev6kFs3aWd2OS0fhO19esptXEeP5CkPHEXDY5zc0V7aeGeTPzlsNcNrnXttOU8dzHa0V7snWbfiYPgqx8Z3zfxKwceyfoclJvGyM/qEd/Vnw5EZbbqCpi4LCBahs9w9Yk5+w/1uLPu7/vsFBc+AMAAA

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-05-14",
      "FRAGMENT":"H4sIAAAAAAAC/72TTXObMBCG7/0VO5r2ZhJJQADdCDiOO+ajhnSaxpmMHMs2NYEERFxPp/+9i7FvPfTUi/Tsakfaj1e/iCz1dEUEqVLtMLvcRWREZLcqVPWsWiIeSJyKdOKTxxFZ1qsDRjIrkgdOL4FanIEfwTjML6mLPInyRWUYZxaQllLrFm6zmwRMlz63Gm6Sa8h0Iwv0M9sw6Qiy6TgPEijrvWpaqNdr1QA+AdwzPncVcGCUfoIoSbP+ErAhxMOPhnNBKcC6boBTg1NYkHaryhImedAuCCR4JbiCM8GtC8s7ZgQP4HHPpZx6Hrie6Xi2g7k6tulcWZRb8Njnf8p6UtZLWYJfqkYbxqLCxqyLptVBo6RWfc84XmRQ22BWTs8vXVnud4zcKrkqi0phVDrz8zwTwDxqGLHaw33d7LAmZv2PBv29P5hh0VdwHvrTE6fUZhbn5ure+Tbbu4fdTbu927rT3ddbUwXbvb/MU1ZVm/jQd6KoWi17iSRrFAmKo5TVppObvmDVB7yotkUzP7yii6NdvJwMotVPfflayqKPe23q92KlGvTHmUhnedY7u2Wmpe5QgATf0aJr5bJUeNJ2yx/qWR+V6QsLPWPBjqtFh633RYJ9CXG/w4kctwDXea9jilM484AxF9d+kA8UhPn9iZLoBOk8CQecxuHdQLNxPEA8nwyQzMP5QMd47IiWO5Wpt67/SUSwEemaDTJ+IXNE3nGSRV39g5Z+f/gDve7N2aUDAAA=",
      "FRAG_NUM":1,
      "GUID":"Pt715lkM__2005142

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-05-14",
      "FRAGMENT":"H4sIAAAAAAAC/72TS3ObMBDH7/0UO5rmZhyJh411I4a4zoSHDek0jTMZ2cguDQEHhBNPp9+9K+PceuipF+1Pq4V96K9fRJRqnhNOqkS5zk0TbciAiC4vZLWRLeEPJEp4MvPI44Cs6/yIkcwOxdGkl0Btk4EXQuBnl9RFnoXZqjKMD+aQlEKpFkLRFJWE606WQIfOBVzHV5CqRhR4yBzDogPIlt71HMr6TTYt1NutbOCmq8Ay9DoGRukFhHGS9j9wwBdH+GwNKQXY1g2Y1DAprEhRbeoVgXgaA7jcZNwcDd3RRJcDDzBxqcVGrmuDRak9ovaYgTVymUUnYxMedfHnkmdlvRYleKVslGGsKpzKtmhaNW2kUFIPzKSYkzoGszPaZxoPKRt9x8gfUuQldoxRya2XZSkHNqGGEck3uK+bZ+yGuf9pOn8bDpZY6BY+rvzpyaTUYba5LXI6jXfy7svXRXC8D7xgPV+8s33gLL4ddofcrfMD019XrRJaIPEWJYLSKEW168ROdyz1rF5k2+I2O+7RZeK+eDlviJLv6nJfikLH7Zv6UOSyQX+U8uQ2S7WzW6dKqA7lRzCP4l0r1qXEk7Zb/5QbddKlx230BJydVpv2RvtCzhY+2jtu2iczxXWpVUzxGj64x8jkV94062nqZ/dnisMzJMvY73Ee+Xc93QZRD9Fy1kO89Jc9neJxIko8y1S+dvodEc4GpGt2yPiArAE54DUWdfUPYvr96Q8e6yaUowMAAA==",
      "FRAG_NUM":1,
      "GUID":"Pt85JrNc__2005142

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-05-14",
      "FRAGMENT":"H4sIAAAAAAAC/8WTUZOaMBDH3/spMpl2pp2KhByo5A1BrVWQAk579jo34YwePS5oSO7KdPrdG0Te+9aX7G+XP8ludvMb0lIu95BAHktbxIdRAweQqn3B+AOrIfkOo5jECw/+GMC82jdaadkhbTAyAbKxBbwQzILMRBPNizC744bRMwFxSaWsQUhFwRmYK1aC9CR6Hw2dd2C+mYK04EfwvhN/AFNagrDi8tH8rDhGAxDOEn+bLL3UA4IWNatBXuzBW2M0HDvgUAnggAnBFsGT4dh1+yyuZy/KKtcbeiUT0jDuuC7vUIha+oJRydrKMcLIQI5h2Rnq93Gd0U4rHxndlzpVrYrXXpalBFguMoyIvYLbSjwBC2H8/8rUGRZtBX3r7u8xQo5l43kTf/x2+uS+3uxUsDTFuHF35nm0lf5sNVFf+UrG51X7N68lbRu9OehW6xaXlB8VPbYFs/aqnlldazdrTjqEtV88Xx0o2S9pnkpatLqTqF6KPRM6HqUkXmdpG1R5KqlUeoygPkcSVdO8ZPpLrfKf7EFe5ssjto7MiHVZbdSZNhYS60ug7ZZg+2J8vSbtNCLdhZ47jDCZen7WkR9kt1fahFeIk03Q4TIKth2tZ1EHUbLoYJMESUcXvb4RSZ9Yys6qfQ+QWAOoxFGzfgg3A/jCRF1U/B9m6c+bv+WKgbNrAwAA",
      "FRAG_NUM":1,
      "GUID":"Pt4rPf6y__2005142FyP+XpH9w3ZuDI/r7y9Z/q6UtCEK8uWnKtPqK",
      "MRN_SRC":"HK1_PRD

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-05-14",
      "FRAGMENT":"H4sIAAAAAAAC/7WTTXObMBCG7/0VO5reamJJgIt1w+A47piPGpKZNMlkhC071AQcJJy6nf73SsY+t5detM++rNCudvUL8UrN14ihOlXFzw0tPqMB4t26FPVKSMQeUJyydOajpwEqmvVRRxIn4keKh4AdSsCPYBrmQ+xpnkX5Y21ZF2aQVlwpCVGz5RLGFK6TCWSq5aURkzQz2peuHlt6IfYA7uZ5sshmoCOkkFCUa/hokSsHw6ZpwcWQBAmAxyhhNr7yiGuOgQfwPOzZIxvb4I3Go7HnEAeeTCrnBGZVU/AK/Eq0yrIea13jpmylClrBlTDlU0yxhV2LODm+/H/sut905Ivg66qshY5KF36eZwzIGFtWLN7hvml3QDB1/lOtfy9VJ1iaAi7te36mGLvEofnKpSNP+LlIduLA23cvuFPD+eRG+cvrF/vTzWYnI7O7loqbZicb3W7d5orX245vTb3C3NSrkFK7+XGvJar98vXsICV+qOG+4qWJ27fNoVyLVutxxtJFnhmxKzLFVadHCelzFOskLyqhv8iu+C5W6jRjPnO0MmXktDq4N0aLGPkaanvLqHMygV6XZiKxbsKFe4wpm/hB3lMQ5vdnSqIzpMsk7HEeh7c9LaZxD/Fy1kOyDJc9neL1jSi+E5l468ybQIwMUNduNevHYA/QQbSybOp/GKXfH/4AQfFxP28DAAA=",
      "FRAG_NUM":1,
      "GUID":"Ptbzf2b7__2005142Tc5268eATeOkevarw8CVt/IBHtARFh3+HfksM",
      "MRN_S

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-05-14",
      "FRAGMENT":"H4sIAAAAAAAC/7WSTY/aMBCG7/0VlrWHVkvANgGBb4GglG2+SkKrbbdaOcSwLiGhscMWVf3vnRC497IXzzPjsT2eef9gUZhljjkuYzPeT4bZGPewaHIly43UmH/HYcxjz8E/ejir8jNkUjsQZ0YGiNiMIidACzcdkAmwF6RPpWXdmKO4EMZo5EUoUeUOeUJXqkCUHI8H9L7b/IAempKRHgpmcx8V1ausNcpUju6Gdp8StK1qNCJowhnlQ9afTsntmdvlRZWJAjmFrI1lPZVQ/1bV2sxrKYxsv8YIIxYZWdROSXfPsE8Z+waZL1LkhSolZMW+k6YJR3RKLCuUr+ixqvdQKxu/4T+gBNWWeGv+8zMjZERttlanh9NyoL+sP0az+0N0/4l+zTdq+uItHJ9MF350aEx7utRGtKOKtjAsGFIhyl0jdu2PZNuLg9Qa3PR8hBADXx2uDjbytxkcC6HavGNdnVQua4iHCY/9NGmDTZYYYRoQAoZ3DG+0yAoJO7rJfsqNuSjE4TZEFpxeVpt0po0FnH52wa45sy9mDuuq1ROBNt+4w5DxmTNPO5q76eOVouAK8SpyO1yG7rojfxF2EK68DqKVu+rokg8dMWIvE/mraRWNOe3hpt4Bg5SHPXyCMamq/A+x/H33D+WMB7ktAwAA",
      "FRAG_NUM":1,
      "GUID":"Pt6k83b6__2005142UivJvI/sVUHOB+mO+K1Wdci9hGEAL09ELOmut",
      "MRN_SRC":"HK1_PRD_A",
      "MRN_TYPE":"STORY

### Real-time News Data Model
The news data appears as JSON in UTF-8 after decompression and assembly of the individual messages.
The Real-time News feed contains the headline, story body text, and associated metadata about the story as a simple group of named values.

Example JSON fields are following:
- ```firstCreated```: UTC timestamp for the first version of the story. Millisecond precision.
- ```headline```: The headline text of the news item.
- ```id```: Uniquely identifies the news item. This is the same value as the GUID in the OMM envelope.
- ```body```: The full body text of the news item (Story Body).

For more detail regarding MRN data model please visit [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item) page. If you want to compare MRN and the legacy N2_UBMS news data model, please visit [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).



In [11]:
print("first 10 headlines\n")
for news in _news_messages[:10]:
    if news["headline"]:
        print(news["headline"])

first 10 headlines

Shenwan Hongyuan Group Co Ltd - 2019 INTERIM REPORT
PDF 1: PALADIN ENERGY LTD (Share Purchase Plan Offer Document)
PDF 1: PALADIN ENERGY LTD (Share Purchase Plan Offer Document)
BREXIT-ZEW SAYS THERE IS STILL HOPE THAT A NO DEAL BREXIT CAN BE AVOIDED
HNX - CKV_20190917155723_Result of transactions of Directors PDMR Ly Chi Duc
HNX - GKM_20190917155723_Board Resolution
Announcement
إعلان
為替相場（対ドル）
삼성 QLED vs LG OLED…불붙는 `8K TV 기술전쟁`


In [12]:
print("first 10 stories\n")
for news in _news_messages[:10]:
    if news["body"]:
        print(news["body"])

first 10 stories

2019 INTERIM REPORT
For best results when printing this announcement, please click on link below:
http://pdf.reuters.com/pdfnews/pdfnews.asp?i=43059c3bf0e37541&u=urn:newsml:reuters.com:20190917:nSN6tXcL2


Please click the above URL to view the article.
Latest version of Adobe Acrobat reader is recommended to view PDF files.  The latest version of the reader can be obtained from http://www.adobe.com/products/acrobat/readstep2.html.

                             PALADIN ENERGY LTD
                                        ACN 061 681 098

17 September 2019


ASX Market Announcements                                              By Electronic Lodgement
Australian Securities Exchange
20 Bridge Street
SYDNEY NSW 2000



NOT FOR RELEASE TO U.S. WIRE SERVICES OR DISTRIBUTION IN THE UNITED STATES.

                         SHARE PURCHASE PLAN OFFER DOCUMENT
Paladin Energy Limited’s (ASX:PDN) (“Paladin” or the “Company”) provides the attached offer
documentation in respect of it

In [13]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install pandas and numpy packages in a current Jupyter kernal\n

# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install numpy

In [14]:
import pandas as pd

In [15]:
df_headlines = pd.DataFrame(_news_messages, columns = ["firstCreated","messageType","provider","language","headline"])

In [16]:
messageType = {
    0: "Unknow",
    1: "Alert",
    2: "First take",
    3: "Subsequent take",
    4: "Correction",
    5: "Corrected",
    6: "Update",
    7: "Deletion",
    8: "Drop due to expiry"
}

In [17]:
df_headlines["messageType"] = [messageType[value] for value in df_headlines["messageType"] if value in messageType ]

In [18]:
df_headlines

,firstCreated,messageType,provider,language,headline
0,2019-09-17T09:01:42.835Z,First take,NS:HIIS,en,Shenwan Hongyuan Group Co Ltd - 2019 INTERIM R...
1,2019-09-17T09:01:42.841Z,First take,NS:SGX,en,PDF 1: PALADIN ENERGY LTD (Share Purchase Plan...
2,2019-09-17T09:01:42.833Z,First take,NS:SGX,en,PDF 1: PALADIN ENERGY LTD (Share Purchase Plan...
3,2019-09-17T09:00:08.808Z,Alert,NS:RTRS,en,BREXIT-ZEW SAYS THERE IS STILL HOPE THAT A NO ...
4,2019-09-17T09:00:41.514Z,First take,NS:HNX,en,HNX - CKV_20190917155723_Result of transaction...
5,2019-09-17T09:00:45.253Z,First take,NS:HNX,en,HNX - GKM_20190917155723_Board Resolution
6,2019-09-17T09:01:37.161Z,First take,NS:BHB,en,Announcement
7,2019-09-17T09:01:38.520Z,First take,NS:BHB,ar,إعلان
8,2019-09-16T15:01:25.665Z,Corrected,NS:KYDO,ja,為替相場（対ドル）
9,2019-09-17T09:01:52.643Z,First take,NS:MKN,ko,삼성 QLED vs LG OLED…불붙는 `8K TV 기술전쟁`


In [19]:
df_story = pd.DataFrame(_news_messages, columns = ["headline","body"])

In [20]:
df_story

,headline,body
0,Shenwan Hongyuan Group Co Ltd - 2019 INTERIM R...,2019 INTERIM REPORT
1,PDF 1: PALADIN ENERGY LTD (Share Purchase Plan...,For best results when printing this announceme...
2,PDF 1: PALADIN ENERGY LTD (Share Purchase Plan...,PALADIN ENERGY LT...
3,BREXIT-ZEW SAYS THERE IS STILL HOPE THAT A NO ...,
4,HNX - CKV_20190917155723_Result of transaction...,\r\n\r\n Double click on the URL below to view...
5,HNX - GKM_20190917155723_Board Resolution,\r\n\r\n Double click on the URL below to view...
6,Announcement,([1]Arabic) ([2]English)\n\nReferences\n\n ...
7,إعلان,([1]عربي) ( [2]إنجليزي)\n\nReferences\n\n ...
8,為替相場（対ドル）,\n１７日\n\n（日本時間）\n\n ０時 １０８円０１－０２銭\n\n １時 １０７円９...
9,삼성 QLED vs LG OLED…불붙는 `8K TV 기술전쟁`,\n 삼성전자와 LG전자가 8K(4K·UHD의 4배 화질) TV의 화질 등 핵심...


## References
* [Refinitiv Elektron SDK Family page](https://developers.refinitiv.com/elektron) on the [Refinitiv Developer Community](https://developers.thomsonreuters.com/) web site.
* [Refinitiv Elektron WebSocket API page](https://developers.refinitiv.com/websocket-api).
* [Developer Webinar Recording: Introduction to Electron WebSocket API](https://www.youtube.com/watch?v=CDKWMsIQfaw).
* [Introduction to Machine Readable News with Elektron WebSocket API](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor](https://github.com/Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor).

For any question related to this example or Elektron WebSocket API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com/spaces/152/websocket-api.html).